In [ ]:
import pandas as pd
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import tqdm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

papers = pd.read_csv('/content/drive/MyDrive/PYTHON/Latent Dirichlet Alg/dataset_informatica_full.csv')
papers = papers.drop(columns = ['keywords', 'Unnamed: 0', 'Unnamed: 0.1','authors', 'language',
                                'difficulty', 'format', 'duration', 'public', 'min_age', 'max_age',
                                'resource_image', 'WikiLink','link','type', 'Upload_Date'], axis=1)
papers = papers[papers['WikiDescription']!="0"]
papers = papers[papers['WikiDescription']!="-"]
papers = papers[papers['WikiDescription']!='']

papers['text_preprocessed']=papers['WikiDescription'].map(lambda x: re.sub('[,.\!?]', '', x))
papers['text_preprocessed']=papers['text_preprocessed'].map(lambda x: x.lower())
#stop_words = stopwords.words('italian')
#dataset['topic'] = 0

papers.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,disciplines,title,description,WikiDescription,text_preprocessed
0,Applied Science/Computer Science,Intro to Scratch,"In this lesson, students will begin exploring ...",Scratch è un ambiente di programmazione gratui...,scratch è un ambiente di programmazione gratui...
1,Applied Science/Computer Science,Intro to Scratch,Students encounter the Scratch programming pla...,"Scratch è adatto a studenti, insegnanti e geni...",scratch è adatto a studenti insegnanti e genit...
2,Applied Science/Computer Science,Intro to Vectors Physics and Augmented Reality,Students learn about video motion capture tech...,"Per realtà aumentata, o realtà mediata dall'el...",per realtà aumentata o realtà mediata dall'ela...
3,Applied Science/Computer Science,Intro to computers,Introduction to Computers - brief videos,"Un computer, è una macchina automatizzata prog...",un computer è una macchina automatizzata progr...
4,Applied Science/Computer Science,Intro to the Zotero API,"In this lesson, you’ll learn how to use python...",Zotero è un software per la gestione di riferi...,zotero è un software per la gestione di riferi...


In [ ]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.text_preprocessed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['scratch', 'un', 'ambiente', 'di', 'programmazione', 'gratuito', 'con', 'un', 'linguaggio', 'di', 'programmazione', 'di', 'tipo', 'grafico']


In [ ]:
stop_words = stopwords.words('italian')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [ ]:
import spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
data_words = remove_stopwords(data_words)
print(data_words)

[['scratch', 'ambiente', 'programmazione', 'gratuito', 'linguaggio', 'programmazione', 'tipo', 'grafico'], ['scratch', 'adatto', 'studenti', 'insegnanti', 'genitori', 'utilizzabile', 'progetti', 'pedagogici', 'intrattenimento', 'spaziano', 'matematica', 'scienza', 'consentendo', 'realizzazione', 'simulazioni', 'visualizzazione', 'esperimenti', 'animazioni', 'musica', 'arte', 'interattiva', 'semplici', 'giochi'], ['realta', 'aumentata', 'realta', 'mediata', 'elaboratore', 'intende', 'arricchimento', 'percezione', 'sensoriale', 'umana', 'mediante', 'informazioni', 'genere', 'manipolate', 'convogliate', 'percepibili', 'cinque', 'sensi'], ['computer', 'macchina', 'automatizzata', 'programmabile', 'grado', 'eseguire', 'complessi', 'calcoli', 'matematici', 'altri', 'tipi', 'elaborazioni', 'dati'], ['zotero', 'software', 'gestione', 'riferimenti', 'bibliografici', 'materiali', 'essi', 'correlati'], ['topologia', 'rete', 'modello', 'geometrico', 'grafo', 'finalizzato', 'rappresentare', 'relazi

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1)]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.35992903899690426


In [ ]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pbar.close()


  0%|          | 0/540 [01:35<?, ?it/s]
/usr/local/lib/python3.9/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)

100%|██████████| 540/540 [2:29:22<00:00, 16.60s/it]


In [ ]:
pd.DataFrame(model_results).to_csv('coherence_results')

In [ ]:
risultati = pd.read_csv('/content/coherence_results')
risultati.head()

,Unnamed: 0,Validation_Set,Topics,Alpha,Beta,Coherence
0,0,75% Corpus,2,0.01,0.01,0.335624
1,1,75% Corpus,2,0.01,0.31,0.298881
2,2,75% Corpus,2,0.01,0.61,0.302463
3,3,75% Corpus,2,0.01,0.9099999999999999,0.317019
4,4,75% Corpus,2,0.01,symmetric,0.288270


In [ ]:
max_coherence = risultati['Coherence'].max()
max_coherence_rows = risultati[risultati['Coherence'] == max_coherence]

print(max_coherence_rows)

     Unnamed: 0 Validation_Set  Topics      Alpha                Beta  \
263         263     75% Corpus      10  symmetric  0.9099999999999999   

     Coherence  
263   0.520186  


In [ ]:
risultati.iloc[263]

Unnamed: 0                       263
Validation_Set            75% Corpus
Topics                            10
Alpha                      symmetric
Beta              0.9099999999999999
Coherence                   0.520186
Name: 263, dtype: object

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='symmetric',
                                           eta=0.909)